In [1]:
import petrovisor as pv

In [2]:
import pandas as pd
import numpy as np

# Authorization

In [ ]:
# url
# discovery_url = r'https://identity.eu1.petrovisor.com'
discovery_url = r'https://identity.us1.petrovisor.com'

# workspace
workspace = 'Workspace Name'

# generate access key (only for the first time, comment the line afterwards)
key = pv.PetroVisor.generate_credentials_key()
# print(f"key : '{key}'")
# save key for later to reuse
# key = ''

pv_api = pv.PetroVisor(workspace = workspace, discovery_url = discovery_url, key = key)
print(pv_api.Api)

# PetroVisor API Swagger

https://api.eu1.petrovisor.com/index.html

https://api.us1.petrovisor.com/index.html

# Examples: Reference Tables

In [4]:
# reference table name
ref_table_name = 'Reference Table Test'

# entity name
entity_name = 'Well 01'

# date
date_1 = '2022-08-04'
date_2 = '2022-08-05'
date_3 = '2022-08-06'

In [5]:
# add reference table definition
pv_api.add_item('ReferenceTableDefinition',{
    'Name': ref_table_name,
    'KeyName': 'x',
    'KeyUnitName': ' ',
    'ValueName': 'y',
    'ValueUnitName': ' ',    
})

{'KeyName': 'x',
 'ValueName': 'y',
 'KeyUnitName': ' ',
 'ValueUnitName': ' ',
 'Modified': '2023-03-21T12:11:59.0025999Z',
 'Created': '2023-03-21T12:11:59.0026Z',
 'Labels': [],
 'Name': 'Reference Table Test'}

In [6]:
# get reference table info
pv_api.get_ref_table_data_info(ref_table_name)

[{'ReferenceTableDefinitionName': 'Reference Table Test',
  'EntityName': 'Well 01',
  'Timestamp': '2022-08-05T00:00:00'},
 {'ReferenceTableDefinitionName': 'Reference Table Test',
  'EntityName': 'Well 01',
  'Timestamp': '2022-08-06T00:00:00'},
 {'ReferenceTableDefinitionName': 'Reference Table Test',
  'EntityName': 'Well 01'},
 {'ReferenceTableDefinitionName': 'Reference Table Test',
  'EntityName': 'Well 01',
  'Timestamp': '2022-08-04T00:00:00'}]

#### Save Reference Table data

In [7]:
# save reference table data (dict)
data = {
    '4': 4,
    '5': 5,
    '6': 6,
    7: 7
}
pv_api.save_ref_table_data(ref_table_name,entity_name,date_1,data)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_1)

{'Data': {'4': 4, '5': 5, '6': 6, '7': 7},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-04T00:00:00'}

In [8]:
# save reference table data (numpy array)
s = np.array([100,111,112,113,114])
pv_api.save_ref_table_data(ref_table_name,entity_name,date_1,s)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_1)

{'Data': {'4': 4, '5': 5, '6': 6, '7': 7},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-04T00:00:00'}

In [9]:
# save reference table data (pandas DataFrame)
df = pd.DataFrame(columns=['A','B'],data=[[1,2],[3,4],[5,6],[7,8]])
pv_api.save_ref_table_data(ref_table_name,entity_name,date_1,df)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_1)

{'Data': {'4': 4, '5': 5, '6': 6, '7': 7},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-04T00:00:00'}

In [10]:
# save reference table data (pandas Series)
s = pd.Series([10,11,12,13,14])
pv_api.save_ref_table_data(ref_table_name,entity_name,date_2,s)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_2)

{'Data': {'0': 10, '1': 11, '2': 12, '3': 13, '4': 14},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-05T00:00:00'}

In [11]:
# save reference table data (list)
s = [45,46,47]
pv_api.save_ref_table_data(ref_table_name,entity_name,date_3,s)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_3)

{'Data': {'0': 45, '1': 46, '2': 47},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-06T00:00:00'}

In [12]:
# save reference table data (list of lists)
s = [[45,12],[46,13],[47,14]]
pv_api.save_ref_table_data(ref_table_name,entity_name,None,s)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,None)

{'Data': {'45': 12, '46': 13, '47': 14},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01'}

In [13]:
# save reference table data (list of tuples)
s = [(445,12),(446,13),(447,14)]
pv_api.save_ref_table_data(ref_table_name,entity_name,date_1,s)

# load reference table data
pv_api.load_ref_table_data(ref_table_name,entity_name,date_1)

{'Data': {'445': 12, '446': 13, '447': 14},
 'ReferenceTableDefinitionName': 'Reference Table Test',
 'EntityName': 'Well 01',
 'Timestamp': '2022-08-04T00:00:00'}

In [14]:
# delete reference table data
pv_api.delete_ref_table_data(ref_table_name,entity_name,date_1)

<Response [200]>